In [7]:
import numpy as np
import os.path as op
from pprint import pformat
from typing import Tuple, Iterator
import time

# EEG utilities
import mne
from mne.preprocessing import ICA, create_eog_epochs
from pyprep.prep_pipeline import PrepPipeline
from autoreject import get_rejection_threshold, validation_curve

# BIDS utilities
from mne_bids import BIDSPath, read_raw_bids
from bids import BIDSLayout
from ..util.io import preprocess.py

SyntaxError: invalid syntax (278740048.py, line 16)

In [11]:
def import_bids_data(bids_root, sub, task, run):
    bids_path = BIDSPath(root = bids_root,
                        subject = sub,
                        task = task,
                        run = run,
                        datatype = 'eeg',
                        )
    print(bids_path)
    raw = read_raw_bids(bids_path, verbose = False)
    raw = raw.pick_types(eeg = True)
    return raw

def read_events(raw):
    events, events_ids = mne.events_from_annotations(raw)
    return events, events_ids

In [34]:
# Constants
BIDS_ROOT = '../../data/bids'
DERIV_ROOT = '../../data/bids/derivatives'

sub = '4'
task = 'pitch'
run = '1'

In [35]:
layout = BIDSLayout(BIDS_ROOT, derivatives = False)
fpaths = layout.get(extension = 'eeg',
                return_type = 'filename')
raw = import_bids_data(BIDS_ROOT, sub, task, run)
events, event_ids = read_events(raw)

# Create virtual EOGs
raw.info

../../data/bids/sub-4/eeg/sub-4_task-pitch_run-1_eeg.vhdr
Used Annotations descriptions: ['100', '150', '200', '250', '50']


/tmp/ipykernel_4949/1160456008.py:9: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['leog', 'reog', 'Ch64', 'Aux1']
  raw = read_raw_bids(bids_path, verbose = False)
/tmp/ipykernel_4949/1160456008.py:9: RuntimeWarning: Not setting position of 1 stim channel found in montage:
['Aux1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(bids_path, verbose = False)


<Info | 12 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 64 EEG
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 64 items (64 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 60.0
 lowpass: 2500.0 Hz
 meas_date: 1924-12-31 10:09:47 UTC
 nchan: 64
 projs: []
 sfreq: 5000.0 Hz
 subject_info: 4 items (dict)
>

In [37]:
len(raw.ch_names)
# raw.info
# raw.ch_names
# import re
# matches = [x for x in raw.ch_names if re.search('Ch', x)]
# matches
# raw.drop_channels(matches)
# raw.ch_na

64

In [33]:
if sub in ['23', '24', '25']:
    print(sub)
len(raw.ch_names)

25


64

In [39]:
layout = BIDSLayout(BIDS_ROOT, derivatives = True)
fpaths = layout.get(scope = 'preprocessing',
                    res = 'hi',
                    suffix='epo',
                    extension = 'fif.gz',
                    return_type = 'filename')
fpaths[3]

'/project2/hcn1/pitch_tracking/scripts/archive/../../data/bids/derivatives/preprocessing/sub-4/sub-4_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz'

In [42]:
epochs = mne.read_epochs(fpaths[3], preload = False)

Reading /project2/hcn1/pitch_tracking/scripts/archive/../../data/bids/derivatives/preprocessing/sub-4/sub-4_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz ...


KeyboardInterrupt: 

In [45]:
len(epochs.ch_names)

61

In [1]:
import numpy as np
import os.path as op

# EEG utilities
import mne

# BIDS utilities
from mne_bids import BIDSPath, read_raw_bids

import pandas as pd

In [2]:
fpath = '../data/raw/letty_subj_6.vhdr'
raw = mne.io.read_raw_brainvision(fpath)
raw.load_data()
raw.set_channel_types({'Aux1': 'stim'})

Extracting parameters from ../data/raw/letty_subj_6.vhdr...
Setting channel info structure...
Reading 0 ... 9212749  =      0.000 ...  1842.550 secs...


/var/folders/hv/vj9_d69170l26hyj9hqqj45h0000gn/T/ipykernel_45747/2487207022.py:4: RuntimeWarning: The unit for channel(s) Aux1 has changed from V to NA.
  raw.set_channel_types({'Aux1': 'stim'})


<RawBrainVision | letty_subj_6.eeg, 64 x 9212750 (1842.5 s), ~4.39 GB, data loaded>

In [3]:
mapping = pd.read_csv('../data/captrak/pitch_tracking_NO_REF.csv')
mapping_dict = {mapping.number[i]: mapping.name[i] for i in range(len(mapping))}
mapping_dict

{'Ch1': 'Fp1',
 'Ch2': 'Fz',
 'Ch3': 'F3',
 'Ch4': 'F7',
 'Ch5': 'FT9',
 'Ch6': 'FC5',
 'Ch7': 'FC1',
 'Ch8': 'C3',
 'Ch9': 'T7',
 'Ch10': 'TP9',
 'Ch11': 'CP5',
 'Ch12': 'CP1',
 'Ch13': 'Pz',
 'Ch14': 'P3',
 'Ch15': 'P7',
 'Ch16': 'O1',
 'Ch17': 'Oz',
 'Ch18': 'O2',
 'Ch19': 'P4',
 'Ch20': 'P8',
 'Ch21': 'TP10',
 'Ch22': 'CP6',
 'Ch23': 'CP2',
 'Ch25': 'C4',
 'Ch26': 'T8',
 'Ch27': 'FT10',
 'Ch28': 'FC6',
 'Ch29': 'FC2',
 'Ch30': 'F4',
 'Ch31': 'F8',
 'Ch32': 'Fp2',
 'Ch33': 'AF7',
 'Ch34': 'AF3',
 'Ch35': 'AFz',
 'Ch36': 'F1',
 'Ch37': 'F5',
 'Ch38': 'FT7',
 'Ch39': 'FC3',
 'Ch40': 'C1',
 'Ch41': 'C5',
 'Ch42': 'TP7',
 'Ch43': 'CP3',
 'Ch44': 'P1',
 'Ch45': 'P5',
 'Ch46': 'PO7',
 'Ch47': 'PO3',
 'Ch48': 'POz',
 'Ch49': 'PO4',
 'Ch50': 'PO8',
 'Ch51': 'P6',
 'Ch52': 'P2',
 'Ch53': 'CPz',
 'Ch54': 'CP4',
 'Ch55': 'TP8',
 'Ch56': 'C6',
 'Ch57': 'C2',
 'Ch58': 'FC4',
 'Ch59': 'FT8',
 'Ch60': 'F6',
 'Ch61': 'AF8',
 'Ch62': 'AF4',
 'Ch63': 'F2',
 'Ch64': 'Iz'}

In [4]:
raw_mapped = raw
raw_mapped.rename_channels(mapping_dict)
raw_mapped.add_reference_channels(ref_channels = ['Cz'])

Location for this channel is unknown; consider calling set_montage() again if needed.


<RawBrainVision | letty_subj_6.eeg, 65 x 9212750 (1842.5 s), ~4.46 GB, data loaded>

In [9]:
raw_located = raw_mapped
dig = mne.channels.read_dig_captrak('../data/captrak/subj_6.bvct')
raw_located = mne.set_bipolar_reference(raw_located, anode = 'Fp1', cathode = 'AF7', ch_name = 'eog1', drop_refs = False)
raw_located = mne.set_bipolar_reference(raw_located, anode = 'Fp2', cathode = 'AF8', ch_name = 'eog2', drop_refs = False)
raw_located.set_channel_types({'eog1': 'eog', 'eog2': 'eog', 'AF7': 'eog', 'AF8': 'eog'})
raw_located.set_montage(dig)

EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=9212750
    Range : 0 ... 9212749 =      0.000 ...  1842.550 secs
Ready.
Added the following bipolar channels:
eog1
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=9212750
    Range : 0 ... 9212749 =      0.000 ...  1842.550 secs
Ready.
Added the following bipolar channels:
eog2


<RawBrainVision | letty_subj_6.eeg, 67 x 9212750 (1842.5 s), ~4.60 GB, data loaded>

In [ ]:
# working condensed version
fpath = '../data/raw/letty_subj_6.vhdr'
raw = mne.io.read_raw_brainvision(fpath)
raw.load_data()
raw.set_channel_types({'Aux1': 'stim'})
mapping = pd.read_csv('../data/captrak/pitch_tracking_NO_REF.csv')
mapping_dict = {mapping.number[i]: mapping.name[i] for i in range(len(mapping))}
raw_mapped = raw
raw_mapped.rename_channels(mapping_dict)
raw_mapped.add_reference_channels(ref_channels = ['Cz'])
raw_located = raw_mapped
dig = mne.channels.read_dig_captrak('../data/captrak/subj_6.bvct')
raw_located = mne.set_bipolar_reference(raw_located, anode = 'Fp1', cathode = 'AF7', ch_name = 'eog1', drop_refs = False)
raw_located = mne.set_bipolar_reference(raw_located, anode = 'Fp2', cathode = 'AF8', ch_name = 'eog2', drop_refs = False)
raw_located.set_channel_types({'eog1': 'eog', 'eog2': 'eog', 'AF7': 'eog', 'AF8': 'eog'})
raw_located.set_montage(dig)

In [ ]:
# latest working version
fpath = '../data/raw/letty_subj_6.vhdr'
raw = mne.io.read_raw_brainvision(fpath)
raw.load_data()
raw.set_channel_types({'Aux1': 'stim'})
raw.info['line_freq'] = 60 # the power line frequency in the building we collected in

mapping = mapping_NO_REF if int(sub) in subs_NO_REF else mapping_REF
raw.rename_channels(mapping)
raw.add_reference_channels(ref_channels = ['Cz'])

dig = mne.channels.read_dig_captrak('../data/captrak/subj_6.bvct')
raw = mne.set_bipolar_reference(raw, anode = 'Fp1', cathode = 'AF7', ch_name = 'eog1', drop_refs = False)
raw = mne.set_bipolar_reference(raw, anode = 'Fp2', cathode = 'AF8', ch_name = 'eog2', drop_refs = False)
raw.set_channel_types({'eog1': 'eog', 'eog2': 'eog', 'AF7': 'eog', 'AF8': 'eog'})
raw.set_montage(dig)

In [9]:
# dig = mne.channels.read_dig_captrak('../data/captrak/sub-5.bvct')
# mapping = {'Ch%s'%i: dig.ch_names[i] for i in range(len(dig.ch_names))}
# del mapping['']

{'Ch0': 'T7',
 'Ch1': 'FC5',
 'Ch2': 'C5',
 'Ch3': 'CP5',
 'Ch4': 'F7',
 'Ch5': 'FT7',
 'Ch6': 'TP7',
 'Ch7': 'P7',
 'Ch8': 'FT9',
 'Ch9': 'TP9',
 'Ch10': 'PO7',
 'Ch11': 'P3',
 'Ch12': 'P5',
 'Ch13': 'C3',
 'Ch14': 'CP3',
 'Ch15': 'O1',
 'Ch16': 'PO3',
 'Ch17': 'Iz',
 'Ch18': 'Oz',
 'Ch19': 'POz',
 'Ch20': 'O2',
 'Ch21': 'PO4',
 'Ch22': 'Pz',
 'Ch23': 'P1',
 'Ch24': 'P2',
 'Ch25': 'CP2',
 'Ch26': 'CPz',
 'Ch27': 'CP1',
 'Ch28': 'Cz',
 'Ch29': 'C1',
 'Ch30': 'FC1',
 'Ch31': 'FC3',
 'Ch32': 'Fz',
 'Ch33': 'F3',
 'Ch34': 'F1',
 'Ch35': 'F5',
 'Ch36': 'AF3',
 'Ch37': 'Fp1',
 'Ch38': 'AFz',
 'Ch39': 'GND',
 'Ch40': 'F2',
 'Ch41': 'Fp2',
 'Ch42': 'AF4',
 'Ch43': 'F4',
 'Ch44': 'F6',
 'Ch45': 'F8',
 'Ch46': 'FC2',
 'Ch47': 'FC4',
 'Ch48': 'FC6',
 'Ch49': 'C2',
 'Ch50': 'C4',
 'Ch51': 'P4',
 'Ch52': 'CP4',
 'Ch53': 'P6',
 'Ch54': 'P8',
 'Ch55': 'PO8',
 'Ch56': 'CP6',
 'Ch57': 'TP10',
 'Ch58': 'TP8',
 'Ch59': 'FT10',
 'Ch60': 'FT8',
 'Ch61': 'T8',
 'Ch62': 'C6'}

In [4]:
# fpath = '../data/raw/letty_subj_2.vhdr'
# raw = mne.io.read_raw_brainvision(fpath)
# raw.ch_names

Extracting parameters from ../data/raw/letty_subj_2.vhdr...
Setting channel info structure...


['Ch1',
 'Ch2',
 'Ch3',
 'Ch4',
 'Ch5',
 'Ch6',
 'Ch7',
 'Ch8',
 'Ch9',
 'Ch10',
 'Ch11',
 'Ch12',
 'Ch13',
 'Ch14',
 'Ch15',
 'Ch16',
 'Ch17',
 'Ch18',
 'Ch19',
 'Ch20',
 'Ch21',
 'Ch22',
 'Ch23',
 'Ch25',
 'Ch26',
 'Ch27',
 'Ch28',
 'Ch29',
 'Ch30',
 'Ch31',
 'Ch32',
 'Ch33',
 'Ch34',
 'Ch35',
 'Ch36',
 'Ch37',
 'Ch38',
 'Ch39',
 'Ch40',
 'Ch41',
 'Ch42',
 'Ch43',
 'Ch44',
 'Ch45',
 'Ch46',
 'Ch47',
 'Ch48',
 'Ch49',
 'Ch50',
 'Ch51',
 'Ch52',
 'Ch53',
 'Ch54',
 'Ch55',
 'Ch56',
 'Ch57',
 'Ch58',
 'Ch59',
 'Ch60',
 'Ch61',
 'Ch62',
 'Ch63',
 'Ch64',
 'Aux1']